#GRADIENT DESCENT ALGORITHM - MultiLayer - Activation Function = Tanh

In [ ]:
from random import random
from random import seed
from ml.activation.Tanh import Tanh

class MultilayerNnClassifier:
    
    def initialize_network_old(self, n_inputs, n_hidden, n_outputs):
        '''
        Initialize a new neural network ready for training. 
        It accepts three parameters, the number of inputs, the number of neurons 
        to have in the hidden layer and the number of outputs.
        '''
        network = list()
        # hidden layer has 'n_hidden' neuron with 'n_inputs' input weights plus the bias
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
        network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
        network.append(output_layer)
        return network
    
    def initialize_network(self, n_inputs, n_hidden, n_outputs):
        '''
        Initialize a new neural network ready for training. 
        It accepts three parameters, the number of inputs, the hidden layers and the number of outputs.
        '''
        network = list()
        h = 0
        for hidden in n_hidden:     
            if(h==0):       
                # hidden layer has 'hidden' neuron with 'n_inputs' input weights plus the bias
                hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(hidden)]
            else:
                # hidden layer has 'hidden' neuron with 'hidden - 1' weights plus the bias
                hidden_layer = [{'weights':[random() for i in range(n_hidden[h-1] + 1)]} for i in range(hidden)]
            network.append(hidden_layer)
            h += 1
        # output layer has 'n_outputs' neuron with 'last hidden' weights plus the bias    
        output_layer = [{'weights':[random() for i in range(n_hidden[-1] + 1)]} for i in range(n_outputs)]
        network.append(output_layer)
        return network
    
    def activate(self, weights, inputs):
        '''
        Calculate neuron activation for an input is the First step of forward propagation
        activation = sum(weight_i * input_i) + bias.
        '''
        activation = weights[-1]  # Bias
        for i in range(len(weights) - 1):
            activation += weights[i] * inputs[i]
        return activation
    
    def forward_propagate(self, network, activation_function, row):
        '''
        Forward propagate input to a network output.
        The function returns the outputs from the last layer also called the output layer.
        '''
        inputs = row
        for layer in network:
            new_inputs = []
            for neuron in layer:
                activation = self.activate(neuron['weights'], inputs)
                neuron['output'] = activation_function.transfer(activation)
                new_inputs.append(neuron['output'])
            inputs = new_inputs
        return inputs
    
    def backward_propagate_error(self, network, activation_function, expected):
        '''
        Backpropagate error and store in neurons.
        
        The error for a given neuron can be calculated as follows:
        
            error = (expected - output) * transfer_derivative(output)
            
        Where expected is the expected output value for the neuron, 
        output is the output value for the neuron and transfer_derivative() 
        calculates the slope of the neuron's output value.
        
        The error signal for a neuron in the hidden layer is calculated as:
        
            error = (weight_k * error_j) * transfer_derivative(output)
            
        Where error_j is the error signal from the jth neuron in the output layer, 
        weight_k is the weight that connects the kth neuron to the current neuron 
        and output is the output for the current neuron.
        '''
        for i in reversed(range(len(network))):
            layer = network[i]
            errors = list()
            if i != len(network) - 1:
                for j in range(len(layer)):
                    error = 0.0
                    for neuron in network[i + 1]:
                        error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
            else:
                for j in range(len(layer)):
                    neuron = layer[j]
                    errors.append(expected[j] - neuron['output'])
            for j in range(len(layer)):
                neuron = layer[j]
                neuron['delta'] = errors[j] * activation_function.transfer_derivative(neuron['output'])
    
    def update_weights(self, network, row, l_rate):
        '''
        Updates the weights for a network given an input row of data, a learning rate 
        and assume that a forward and backward propagation have already been performed.
        
            weight = weight + learning_rate * error * input
            
        Where weight is a given weight, learning_rate is a parameter that you must specify, 
        error is the error calculated by the back-propagation procedure for the neuron and 
        input is the input value that caused the error.
        '''
        for i in range(len(network)):
            inputs = row[:-1]
            if i != 0:
                inputs = [neuron['output'] for neuron in network[i - 1]]
            for neuron in network[i]:
                for j in range(len(inputs)):
                    neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
                neuron['weights'][-1] += l_rate * neuron['delta']
    
    def train_network(self, network, activation_function, train, l_rate, n_epoch, n_outputs):
        '''
        Train a network for a fixed number of epochs.
        The network is updated using stochastic gradient descent.
        '''
        for epoch in range(n_epoch + 1):
            sum_error = 0
            for row in train:
                # Calculate Loss
                outputs = self.forward_propagate(network, activation_function, row)
                expected = [0 for i in range(n_outputs)]
                expected[row[-1]] = 1  # Bias
                sum_error += sum([(expected[i] - outputs[i]) ** 2 for i in range(len(expected))])
                self.backward_propagate_error(network, activation_function, expected)
                self.update_weights(network, row, l_rate)
            if (epoch % 100 == 0):    
                print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error/float(len(train))))
    
    def predict(self, network, activationFunction, row):
        '''
        Make a prediction with a network.
        We can use the output values themselves directly as the probability of a pattern belonging to each output class.
        It may be more useful to turn this output back into a crisp class prediction. 
        We can do this by selecting the class value with the larger probability. 
        This is also called the arg max function.
        '''
        outputs = self.forward_propagate(network, activationFunction, row)
        return outputs.index(max(outputs))
    
    def back_propagation(self, train, test, l_rate, n_epoch, n_hidden, activationFunction):
        '''
        Backpropagation Algorithm With Stochastic Gradient Descent
        '''
        n_inputs = len(train[0]) - 1
        n_outputs = len(set([row[-1] for row in train]))
        network = self.initialize_network(n_inputs, n_hidden, n_outputs)
        self.train_network(network, activationFunction, train, l_rate, n_epoch, n_outputs)
        predictions = list()
        for row in test:
            prediction = self.predict(network, activationFunction, row)
            predictions.append(prediction)
        return(predictions)

if __name__ == '__main__':    
    
    seed(1)
    mlp = MultilayerNnClassifier()
    activationFunction = Tanh()
    network = mlp.initialize_network(2, [10,5], 2)
    for layer in network:
        print(layer)
       
    # Test forward_propagate
    print("Test Forward")
    row = [1, 0, None]
    output = mlp.forward_propagate(network, activationFunction, row)
    print(output)
    
    # Test backward_propagate_error
    print("Test backpropagation of error")
    network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
            [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
    expected = [0, 1]
    mlp.backward_propagate_error(network, activationFunction, expected)
    for layer in network:
        print(layer)
      
    # Test training backprop algorithm
    print("Test training backprop algorithm")
    seed(1)
    dataset = [[2.7810836, 2.550537003, 0],
        [1.465489372, 2.362125076, 0],
        [3.396561688, 4.400293529, 0],
        [1.38807019, 1.850220317, 0],
        [3.06407232, 3.005305973, 0],
        [7.627531214, 2.759262235, 1],
        [5.332441248, 2.088626775, 1],
        [6.922596716, 1.77106367, 1],
        [8.675418651, -0.242068655, 1],
        [7.673756466, 3.508563011, 1]]
    n_inputs = len(dataset[0]) - 1
    n_outputs = len(set([row[-1] for row in dataset]))
    network = mlp.initialize_network(n_inputs, [2], n_outputs)
    mlp.train_network(network, activationFunction, dataset, 0.5, 20, n_outputs)    
    for layer in network:
        print(layer)
    for row in dataset:
        prediction = mlp.predict(network, activationFunction, row)
        print('Expected=%d, Got=%d' % (row[-1], prediction))  

#Batch GRADIENT DESCENT 

In [ ]:
import numpy as np
import random
from sklearn.datasets.samples_generator import make_regression 
import pylab
from scipy import stats

def gradient_descent_2(alpha, x, y, numIterations):
    m = x.shape[0] # number of samples
    theta = np.ones(2)
    x_transpose = x.transpose()
    for iter in range(0, numIterations):
        hypothesis = np.dot(x, theta)
        loss = hypothesis - y
        J = np.sum(loss ** 2) / (2 * m)  # cost
        print("iter %s | J: %.3f" % (iter, J))  
        gradient = np.dot(x_transpose, loss) / m         
        theta = theta - alpha * gradient  # update
    return theta

if __name__ == '__main__':

    x, y = make_regression(n_samples=100, n_features=1, n_informative=1, 
                        random_state=0, noise=35) 
    m, n = np.shape(x)
    x = np.c_[ np.ones(m), x] # insert column
    alpha = 0.01 # learning rate
    theta = gradient_descent_2(alpha, x, y, 1000)

    # plot
    for i in range(x.shape[1]):
        y_predict = theta[0] + theta[1]*x 
    pylab.plot(x[:,1],y,'o')
    pylab.plot(x,y_predict,'k-')
    pylab.show()
    print("Done!")

iter 0 | J: 1604.873

iter 1 | J: 1586.636

iter 2 | J: 1568.768

iter 3 | J: 1551.261
...

iter 997 | J: 699.300

iter 998 | J: 699.300

iter 999 | J: 699.300

theta = [ -2.84837957  43.20234847]

In [2]:
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df_Data_Normal = pd.read_csv("Data_Normal.txt")
print(df_Data_Normal.head())
df_Data_Normal.shape

   Height     Weight    Male
0  151.765  47.825606      1
1  139.700  36.485807      0
2  136.525  31.864838      0
3  156.845  53.041915      1
4  145.415  41.276872      0


(60, 3)

#Glass

In [7]:
import csv
with open('Glass.txt', 'r') as infile, open('Glass.csv', 'w') as outfile:
      stripped = (line.strip() for line in infile)
      lines = (line.split("\t") for line in stripped if line)
      writer = csv.writer(outfile)
      writer.writerows(lines)

In [9]:
df_Glass = pd.read_csv("Glass.csv")

#Normalize

In [10]:
from sklearn import preprocessing
import numpy as np
d_Glass = preprocessing.normalize(df_Glass)
scaled_df_Normal_Glass = pd.DataFrame(d_Glass, columns=['1','2','3','4','5','6','7','8','9','10','11','12'])
scaled_df_Normal_Glass.head()

,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.5,0.0,0.500000,0.000000,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.500000,0.500000,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0
2,0.0,0.5,0.0,0.500000,0.000000,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0
3,0.0,0.5,0.0,0.500000,0.000000,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.377964,0.377964,0.0,0.755929,0.377964,0.0,0.0,0.0,0.0


In [ ]:
from sklearn import preprocessing
import numpy as np
d = preprocessing.normalize(df_Data_Normal)
scaled_df_Normal = pd.DataFrame(d, columns=['Height','Weight','Male'])
scaled_df_Normal.head()

,Height,Weight,Male
0,0.953744,0.300553,0.006284
1,0.967546,0.252696,0.000000
2,0.973827,0.227291,0.000000
3,0.947280,0.320351,0.006040
4,0.961995,0.273068,0.000000


In [13]:
X_Glass = scaled_df_Normal_Glass.loc[:, ['1','2','3','4','5','6','7','8','9','10','11']]
y_Glass = scaled_df_Normal_Glass.loc[:, ['12']]

In [ ]:
X = scaled_df_Normal.Height
y = scaled_df_Normal.Weight

#Split 30% Test and 70% Train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [14]:
from sklearn.model_selection import train_test_split
X_train_G, X_test_G, y_train_G, y_test_G = train_test_split(X_Glass, y_Glass, test_size=0.30, random_state=42)

In [15]:
def ssr_gradient(x, y, b):
    res = b[0] + b[1] * x - y
    return res.mean(), (res * x).mean()  # .mean() is a method of np.ndarray

In [ ]:
gradient_descent(
ssr_gradient, x, y, start=[0.5, 0.5], learn_rate=0.0008,
n_iter=100_000
)

In [ ]:
gradient_descent(
ssr_gradient, X_Glass, y_Glass, start=[0.5, 0.5], learn_rate=0.0008,
n_iter=100_000
)

#Metric

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.67      1.00      0.80         2
     class 1       0.00      0.00      0.00         1
     class 2       1.00      0.50      0.67         2

    accuracy                           0.60         5
   macro avg       0.56      0.50      0.49         5
weighted avg       0.67      0.60      0.59         5

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true_Glass, y_pred_Glass)

In [ ]:
target_names_Glass = ['1','2','3','4','5']
print(classification_report(y_true_Glass, y_pred_Glass, target_names=target_names_Glass))

              precision    recall  f1-score   support

     class 0       0.87      1.00      0.80         2
     class 1       0.00      0.00      0.00         1
     class 2       1.00      0.70      0.84         2

    accuracy                           0.87         5
   macro avg       0.76      0.50      0.79         5
weighted avg       0.77      0.60      0.80         5